# <font color='blue'><u>Estudo de Caso</u></font>

## <font color='blue'>Pré Processamento de Dados Para E-Commerce Analytics</font>

#### Obs: Estaremos utilizando a mesma base de dados para trabalhar em um grande projeto de Ciência de Dados distribuído em 3 capítulos:

- Capítulo 4 (Análise Exploratória de Dados) 
- Capítulo 5 (Engenharia de Atributos)     
- Capítulo 6 (Pré-Processamento de Dados)   -> <strong>Atual</strong>

## Definição do Problema

Uma empresa internacional de comércio eletrônico (E-commerce) que vende produtos eletrônicos deseja descobrir informações importantes de seu banco de dados de clientes.

Os produtos ficam armazenados em um armazém na sede da empresa. Após concluir a compra no web site da empresa, o cliente recebe o produto em casa, em qualquer parte do mundo.  Os  produtos  são  enviados  de  Navio,  Avião  ou  Caminhão,  dependendo  da  região  de entrega.

Em cada compra o cliente pode receber um desconto dependendo do peso do produto comprado. Cada cliente pode fazer chamadas ao suporte da empresa no caso de dúvidas ou problemas e após receber o produto o cliente pode deixar uma avaliação sobre a experiência de compra. O único dado pessoal sobre o cliente que está disponível é o gênero.

Nosso trabalho neste <u>Estudo de Caso</u> é explorar os dados, compreender como estão organizados, detectar eventuais problemas e analisar os dados por diferentes perspectivas.

Trabalharemos com dados fictícios que representam dados reais de uma empresa de E-Commerce. Os dados estão disponíveis na pasta "dados". <br><br><br>

## Dicionário de Dados<br><br>

<table border="2">
  <tr>
    <th>Nome da Coluna</th>
    <th>Tipo de Dado</th>
    <th>Descrição</th>
  </tr>
  <tr>
    <td>ID</td>
    <td>Int</td>
    <td>ID exclusivo de cada transação.</td>
  </tr>
  <tr>
    <td>corredor_armazem</td>
    <td>Character(1)</td>
    <td>Letra do corredor do armazém onde o produto está armazenado.</td>
  </tr>
  <tr>
    <td>modo_envio</td>
    <td>String</td>
    <td>Modo de envio do produto.</td>
  </tr>
  <tr>
    <td>numero_chamadas_cliente</td>
    <td>Int</td>
    <td>Número de vezes que o cliente acionou o suporte da empresa.</td>
  </tr>
  <tr>
    <td>avaliacao_cliente</td>
    <td>Int</td>
    <td>Avaliação do cliente sobre a experiência de compra.</td>
  </tr>
  <tr>
    <td>custo_produto</td>
    <td>Int</td>
    <td>Custo do produto.</td>
  </tr>
  <tr>
    <td>compras_anteriores</td>
    <td>Int</td>
    <td>Número de vezes que o cliente fez uma compra na empresa.</td>
  </tr>
  <tr>
    <td>prioridade_produto</td>
    <td>String</td>
    <td>Prioridade de entrega do produto.</td>
  </tr>
  <tr>
    <td>genero</td>
    <td>Character(1)</td>
    <td>Gênero do cliente (F ou M).</td>
  </tr>
  <tr>
    <td>desconto</td>
    <td>Int</td>
    <td>Desconto concedido na compra do produto.</td>
  </tr>
  <tr>
    <td>peso_gramas</td>
    <td>Int</td>
    <td>Peso do produto.</td>
  </tr>
  <tr>
    <td>entregue_no_prazo</td>
    <td>Character(1)</td>
    <td>Se o produto foi entregue no prazo, sendo 0 (não foi entregue no prazo) ou 1 (foi entregue no prazo).</td>
  </tr>
</table><br><br><br><br>

In [1]:
# Imports
import sklearn
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

In [2]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

numpy  : 1.26.4
pandas : 2.2.1
sklearn: 0.24.2



## <br>Carregando o Dataset (Gerado ao Final do Estudo de Caso 4)

In [3]:
# Carrega o dataset
df = pd.read_csv('dados/df_eng.csv', index_col = 0)

df.shape

(10643, 16)

In [4]:
# Visualiza
df.head()

,ID,corredor_armazem,modo_envio,numero_chamadas_cliente,avaliacao_cliente,custo_produto,compras_anteriores,prioridade_produto,genero,desconto,peso_gramas,entregue_no_prazo,performance_prioridade_envio,performance_modo_envio,faixa_desconto,performance_faixa_desconto
0,1,D,Aviao,4,2,177,3,baixa,F,44,1233,1,Não Houve Atraso,Não Houve Atraso,Desconto Acima da Media,Entrega no Prazo com Desconto Acima da Media
1,2,F,Aviao,4,5,216,2,baixa,M,59,3088,1,Não Houve Atraso,Não Houve Atraso,Desconto Acima da Media,Entrega no Prazo com Desconto Acima da Media
2,3,A,Aviao,2,2,183,4,baixa,M,48,3374,1,Não Houve Atraso,Não Houve Atraso,Desconto Acima da Media,Entrega no Prazo com Desconto Acima da Media
3,4,B,Aviao,3,3,176,4,media,M,10,1177,1,Não Houve Atraso,Não Houve Atraso,Desconto Abaixo da Media,Entrega no Prazo com Desconto Abaixo da Media
4,5,C,Aviao,2,2,184,3,media,F,46,2484,1,Não Houve Atraso,Não Houve Atraso,Desconto Acima da Media,Entrega no Prazo com Desconto Acima da Media


In [5]:
# Info
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10643 entries, 0 to 10999
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   ID                            10643 non-null  int64 
 1   corredor_armazem              10643 non-null  object
 2   modo_envio                    10643 non-null  object
 3   numero_chamadas_cliente       10643 non-null  int64 
 4   avaliacao_cliente             10643 non-null  int64 
 5   custo_produto                 10643 non-null  int64 
 6   compras_anteriores            10643 non-null  int64 
 7   prioridade_produto            10643 non-null  object
 8   genero                        10643 non-null  object
 9   desconto                      10643 non-null  int64 
 10  peso_gramas                   10643 non-null  int64 
 11  entregue_no_prazo             10643 non-null  int64 
 12  performance_prioridade_envio  10643 non-null  object
 13  performance_modo_envi

# <br><br> Label Encoding

### <br>Método 1

In [6]:
# Verificando value_counts de prioridade_produto (Variável categórica ordinal)
df.prioridade_produto.value_counts()

prioridade_produto
baixa    5174
media    4587
alta      882
Name: count, dtype: int64